In [1]:
import mlflow.pytorch
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.models import DeepGraphInfomax
from tqdm import tqdm
import pandas as pd
import graph_reinforcement_learning_using_blockchain_data as grl
from graph_reinforcement_learning_using_blockchain_data import config

config.load_dotenv()

2025-05-04 10:25:37.381 | INFO     | graph_reinforcement_learning_using_blockchain_data.config:<module>:12 - PROJ_ROOT path is: /Users/liamtessendorf/Programming/Uni/2_Master/4_FS25_Programming/graph-reinforcement-learning-using-blockchain-data


True

## Load Data

In [2]:
dataset = torch.load(config.FLASHBOTS_Q2_DATA_DIR / "state_graphs.pt", weights_only=False)

In [3]:
max_feats_len = 0
for vals in dataset.values():
    for graphs in vals:
        if graphs.x.shape[1] > max_feats_len:
            max_feats_len = graphs.x.shape[1]

In [4]:
for vals in dataset.values():
    for graphs in vals:
        graphs.x = grl.pad_features(graphs.x, max_feats_len)

In [5]:
data_list = [d for sublist in dataset.values() for d in sublist]

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(
    data_list, [0.8, 0.2], generator=generator
)

In [6]:
train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True, exclude_keys=["account_mapping"], drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)
data_loader = DataLoader(
    data_list, batch_size=256, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)

## Train DGI

In [14]:
summary = lambda z, *_, batch=None, **__: global_mean_pool(z, batch)


def corruption(data):
    """
    Corrupts the graph data by shuffling node features.
    Accepts a Data/Batch object and returns a corrupted Data/Batch object.
    """
    corrupted_data = data.clone()  # Create a copy to avoid modifying the original data
    num_nodes_total = data.x.size(0)
    # Generate a permutation of node indices within the batch
    perm = torch.randperm(num_nodes_total, device=data.x.device)
    corrupted_data.x = data.x[perm]  # Apply the permutation to shuffle features
    return corrupted_data

In [15]:
num_node_features = max_feats_len
dim_global_features = 0
hidden_channels = 128

device = torch.device("mps")
enc = grl.SAGEEncoder(num_node_features, hidden_channels).to(device)
dgi = DeepGraphInfomax(
    hidden_channels=hidden_channels, encoder=enc, summary=summary, corruption=corruption
).to(device)

In [11]:
enc = grl.pretrain_dgi(train_loader, dgi, device, epochs=10)

2025/05/03 11:26:57 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
 10%|█         | 1/10 [02:00<18:06, 120.70s/it]

[DGI] epoch 01  loss=0.3962


 20%|██        | 2/10 [04:32<18:33, 139.21s/it]

[DGI] epoch 02  loss=0.1031


 30%|███       | 3/10 [07:46<19:08, 164.11s/it]

...

[DGI] epoch 09  loss=0.0704


100%|██████████| 10/10 [35:01<00:00, 210.11s/it]

[DGI] epoch 10  loss=0.0594



2025/05/03 12:02:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DGI 128 at: http://127.0.0.1:8080/#/experiments/330930495026013213/runs/dc95b44ca9ad4acdbe557b133f6b6653
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/330930495026013213


## Train GraphSAGE Classifier using frozen DGI embeddings

In [11]:
model_uri = "mlflow-artifacts:/330930495026013213/1f3f3d97f6a14569b9f2f285c5a98bb4/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)
enc = model.encoder

In [16]:
for p in enc.parameters():
    p.requires_grad = False

model = grl.GraphSAGEClassifier(enc, hidden=128, num_classes=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

In [18]:
model, embeddings = grl.run_experiment(
    "Unsupervised GraphSAGE with DGI",
    10,
    model,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    return_embeddings=True,
)

2025/05/04 10:31:53 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 starts
🏃 View run powerful-whale-206 at: http://127.0.0.1:8080/#/experiments/132032870842317128/runs/82a148189d3648e0b92db81d4eb3821e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/132032870842317128


RuntimeError: linear(): input and weight.T shapes cannot be multiplied (256x128 and 256x128)

## Create Embeddings from pre-trained GNN for downstream tasks

In [19]:
model_uri = "mlflow-artifacts:/132032870842317128/ba4bedca3761489498f42f57c3986752/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)

In [20]:
criterion = torch.nn.CrossEntropyLoss()
_, _, embeddings = grl.test(model, data_loader, criterion, device, return_embeddings=True)

In [21]:
emb = {trx_id: emb.cpu().detach().numpy().tolist() for trx_id, emb in embeddings.items()}

In [22]:
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

In [23]:
df_embeddings.to_csv(
    config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings_pre_trained_128.csv", index=False
)

## Create Embeddings from DGI encoder for downstream tasks

In [13]:
model_uri = "mlflow-artifacts:/330930495026013213/dc95b44ca9ad4acdbe557b133f6b6653/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)

In [14]:
model.eval()
unsup_list, y_list = [], []
device = torch.device("mps")
embeddings = {}
with torch.no_grad():
    for data in tqdm(data_loader):
        data = data.to(device)

        z_nodes = model.encoder(data)
        unsup_emb = global_mean_pool(z_nodes, data.batch)

        mapping = {trx_id: emb for trx_id, emb in zip(data.trx_id, unsup_emb)}
        embeddings.update(mapping)

        unsup_list.append(unsup_emb.cpu())
        y_list.append(data.y.cpu())

unsup_X = torch.cat(unsup_list).numpy()
y = torch.cat(y_list).numpy()

100%|██████████| 583/583 [10:27<00:00,  1.08s/it]


In [15]:
emb = {trx_id: emb.cpu().detach().numpy().tolist() for trx_id, emb in embeddings.items()}
df_embeddings = pd.DataFrame({"transactionHash": emb.keys(), "embeddings": emb.values()})

df_embeddings.to_csv(config.FLASHBOTS_Q2_DATA_DIR / "state_embeddings_dgi_128.csv", index=False)